
# SVM & MLP：訓練過程、損失函數與參數更新（對應白板說明）

本筆記整理白板上的重點（MLP、Loss function、HW1：推導 SVM 與 MLP 的「訓練過程」、以及程式中如何實現 \(W^\*=W+\Delta W\)），提供對應的數學式與簡潔推導，並附上可執行的最小化程式骨架（不依賴資料集，只示範更新規則）。



## 0. 共同核心：梯度下降與參數更新

- 目標：最小化損失 \(L(W)\)（可含正則化）。  
- 基本更新式（對應白板的 \(W^\* = W + \Delta W\)）：  
\[
\boxed{\;\Delta W = -\eta \nabla_W L(W)\;,\qquad W \leftarrow W + \Delta W = W - \eta \nabla_W L(W)\;}
\]

**參數意義**  
\(\eta\)：學習率（步長）；\(\nabla_W L\)：對參數 \(W\) 的梯度；\(L\)：損失函數（見各模型）。

> 小批次（mini-batch）時，\(\nabla_W L\) 以該批資料的平均梯度近似；若含 L2 正則化 \(\frac{\lambda}{2}\|W\|^2\)，則梯度多出 \(\lambda W\)。



## 1. SVM 的訓練過程（線性、soft-margin、二分類）

- 分類面：\(f(x) = w^\top x + b\)，標籤 \(y_i \in \{-1,+1\}\)。  
- 合頁損失（hinge）+ L2 正則化（常用於 SGD）：  
\[
\boxed{\;L(w,b)=\frac{\lambda}{2}\|w\|^2+\frac{1}{n}\sum_{i=1}^{n}\max\bigl(0,\;1-y_i(w^\top x_i+b)\bigr)\;}
\]

**次梯度（以單一樣本 \( (x_i,y_i)\) 說明）**：
\[
\partial_w L=
\begin{cases}
\lambda w & \text{若 } y_i(w^\top x_i+b)\ge 1\\[2pt]
\lambda w - y_i x_i & \text{若 } y_i(w^\top x_i+b)< 1
\end{cases},
\qquad
\partial_b L=
\begin{cases}
0 & \text{若 } y_i(w^\top x_i+b)\ge 1\\
- y_i & \text{若 } y_i(w^\top x_i+b)< 1
\end{cases}
\]

**SGD 更新（對應 \(W^\* = W + \Delta W\)）**：
\[
\boxed{
\begin{aligned}
\text{若 } y_i(w^\top x_i+b)\ge 1:&\quad w\leftarrow (1-\eta\lambda)w,\; b\leftarrow b\\
\text{否則 }&:\quad w\leftarrow (1-\eta\lambda)w+\eta y_i x_i,\; b\leftarrow b+\eta y_i
\end{aligned}}
\]

> 意義：當樣本在「間隔內或被誤分」時（margin < 1）才用 \(y_i x_i\) 方向拉回決策面；否則只做權重衰減以實現 L2 正則化。


In [ ]:

# 迷你版：線性 SVM 的單步 SGD 更新（示例）
# x: (d,), y in {-1, +1}
# w: (d,), b: scalar

def svm_sgd_step(w, b, x, y, eta=1e-2, lam=1e-3):
    margin = y * (w @ x + b)
    if margin >= 1.0:
        # 只做 L2 正則衰減
        w = (1 - eta * lam) * w
        # b 無變化
    else:
        # 合頁損失發生，推一把
        w = (1 - eta * lam) * w + eta * y * x
        b = b + eta * y
    return w, b

# 使用範例（隨機向量，僅示範 API）：
if __name__ == "__main__":
    import numpy as np
    rng = np.random.default_rng(0)
    w = rng.normal(size=5)
    b = 0.0
    x = rng.normal(size=5)
    y = 1 if rng.random() > 0.5 else -1
    w_new, b_new = svm_sgd_step(w, b, x, y)
    print("w_new:", w_new)
    print("b_new:", b_new)



## 2. MLP（Multiple Layer Perceptron）訓練過程（以 1 隱藏層為例）

### 前向傳播
\[
\begin{aligned}
a^{(1)} &= W_1 x + b_1,\quad h=\sigma(a^{(1)})\\
z &= W_2 h + b_2,\quad \hat{y}=g(z)
\end{aligned}
\]
- \(\sigma(\cdot)\)：隱藏層活化（ReLU、tanh…）  
- \(g(\cdot)\)：輸出層映射（分類常用 softmax；回歸多用恒等）

### 常見損失
- 多類分類（softmax + 交叉熵）：
\[
\boxed{L = -\frac{1}{n}\sum_{i=1}^{n}\sum_{k} y_{ik}\,\log \hat{y}_{ik}}
\]
- 回歸（均方誤差 MSE）：
\[
\boxed{L=\frac{1}{2n}\sum_{i=1}^{n}\|\hat{y}_i-y_i\|^2}
\]

### 反向傳播（以 softmax+CE 為例）
\[
\begin{aligned}
\delta^{(2)} &= \hat{y}-y \quad (\text{輸出層誤差})\\
\nabla_{W_2} L &= \delta^{(2)} h^\top,\qquad \nabla_{b_2} L=\delta^{(2)}\\[4pt]
\delta^{(1)} &= \bigl(W_2^\top \delta^{(2)}\bigr)\odot \sigma'(a^{(1)})\\
\nabla_{W_1} L &= \delta^{(1)} x^\top,\qquad \nabla_{b_1} L=\delta^{(1)}
\end{aligned}
\]
> 其中 \(\odot\) 為逐元素乘，\(\sigma'\) 為活化導數（ReLU 的導數為指示函數 \(1_{a^{(1)}>0}\)）。

### 參數更新（對應 \(W^\*=W+\Delta W\)）
\[
\boxed{
\begin{aligned}
W_2 &\leftarrow W_2 - \eta\,\nabla_{W_2}L,\quad b_2 \leftarrow b_2 - \eta\,\nabla_{b_2}L\\
W_1 &\leftarrow W_1 - \eta\,\nabla_{W_1}L,\quad b_1 \leftarrow b_1 - \eta\,\nabla_{b_1}L
\end{aligned}}
\]
若含 L2 正則化 \(\frac{\lambda}{2}(\|W_1\|^2+\|W_2\|^2)\)，則 \(\nabla_{W_\ell}L\) 額外加上 \(\lambda W_\ell\)。


In [ ]:

# 迷你版：一隱藏層 MLP（softmax+CE）的單步反向傳播與參數更新骨架
import numpy as np

def softmax(z):
    z = z - z.max(axis=0, keepdims=True)
    e = np.exp(z)
    return e / e.sum(axis=0, keepdims=True)

def relu(a):
    return np.maximum(0.0, a)

def relu_grad(a):
    return (a > 0).astype(a.dtype)

def cross_entropy(probs, y_onehot):
    # probs: (C, B), y_onehot: (C, B)
    # return scalar平均損失
    eps = 1e-12
    return -np.mean(np.sum(y_onehot * np.log(probs + eps), axis=0))

def mlp_forward(W1, b1, W2, b2, x):
    # x: (d, B)
    a1 = W1 @ x + b1  # (h, B)
    h = relu(a1)      # (h, B)
    z = W2 @ h + b2   # (C, B)
    yhat = softmax(z) # (C, B)
    cache = (x, a1, h, z, yhat)
    return yhat, cache

def mlp_backward(W1, b1, W2, b2, cache, y_onehot, lam=0.0):
    x, a1, h, z, yhat = cache
    B = x.shape[1]

    # 輸出層誤差
    delta2 = (yhat - y_onehot) / B                    # (C, B)

    # 梯度（含 L2 正則）
    dW2 = delta2 @ h.T + lam * W2                     # (C, h)
    db2 = delta2.sum(axis=1, keepdims=True)           # (C, 1)

    delta1 = (W2.T @ delta2) * relu_grad(a1)          # (h, B)
    dW1 = delta1 @ x.T + lam * W1                     # (h, d)
    db1 = delta1.sum(axis=1, keepdims=True)           # (h, 1)

    return dW1, db1, dW2, db2

def mlp_update(W1, b1, W2, b2, grads, eta=1e-2):
    dW1, db1, dW2, db2 = grads
    W1 = W1 - eta * dW1
    b1 = b1 - eta * db1
    W2 = W2 - eta * dW2
    b2 = b2 - eta * db2
    return W1, b1, W2, b2

# 使用範例（隨機資料，僅示範 API 與張量形狀）：
if __name__ == "__main__":
    rng = np.random.default_rng(0)
    d, h, C, B = 8, 10, 3, 4  # input dim, hidden, classes, batch size
    W1 = rng.normal(scale=0.1, size=(h, d)); b1 = np.zeros((h, 1))
    W2 = rng.normal(scale=0.1, size=(C, h)); b2 = np.zeros((C, 1))

    x = rng.normal(size=(d, B))                 # (d, B)
    y_idx = rng.integers(0, C, size=(B,))       # (B,)
    y_onehot = np.zeros((C, B)); y_onehot[y_idx, np.arange(B)] = 1.0

    yhat, cache = mlp_forward(W1, b1, W2, b2, x)
    loss = cross_entropy(yhat, y_onehot)
    grads = mlp_backward(W1, b1, W2, b2, cache, y_onehot, lam=1e-3)
    W1, b1, W2, b2 = mlp_update(W1, b1, W2, b2, grads, eta=1e-1)

    print("loss (one step):", float(loss))



## 3. Loss function 與「谷底圖」的連結

沿著 \(-\eta \nabla L\) 方向，每步往低處前進；\(\eta\) 太大可能震盪、太小則收斂慢。實務常用：
- **學習率排程**（衰減 / 餘弦 / 週期性）  
- **Momentum / Nesterov**、**Adam**、**RMSProp** 等自適應法  
- **Early Stopping**、**權重衰減（L2）**、**Dropout** 等正則化



## 4. 程式實作對照（Checklist）

1. **前向**：依模型計算 \(\hat{y}\) 與 \(L\)。  
2. **反向**：依上式求所有 \(\nabla L\)。  
3. **更新**：依 \(W \leftarrow W - \eta \nabla L\)（即白板的 \(W^\*=W+\Delta W\) 且 \(\Delta W=-\eta\nabla L\)）。  
4. **迭代**：直到收斂或達到訓練輪數。
